# Test Chromadb Scripts

In [21]:
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
import os
load_dotenv()

# Preparing embedding models for embedding the chroma 
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv("OPENAI_API_KEY"),
                model_name="text-embedding-ada-002",
            )

# chroma_client = chromadb.HttpClient(host='localhost', port=8000)
client = chromadb.HttpClient(host='localhost', port=8000)



In [2]:
chroma_client.heartbeat()

1700200571334541723

In [19]:
for i in chroma_client.list_collections():
    print(i)

name='Fianancial_News' id=UUID('cb79b7d8-7bcf-49ce-babf-d5b4ee0a4c27') metadata={'News_category': 'Gulf', 'News_source': 'Reuters_News'}


## Data exploration

In [1]:
import pandas as pd

data = pd.read_csv("raw_data/News Articles collection_file_7.csv")

In [2]:
data.columns

Index(['_id', 'source_id', 'source_name', 'sentiment', 'summary',
       'description', 'title', 'content', 'publishedAt_str', 'publishedAt',
       'overall_polarity', 'subjectivity_score', 'sentiment_tag'],
      dtype='object')

In [3]:
columns=["_id","source_id","source_name","title","content","publishedAt_str"]
data = data[columns]

In [4]:
data.shape

(1000, 6)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1200,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)

In [16]:
docs=[]
selected_column = 'content'

for index, row in data.iterrows():
    metadata_dict = {}
    selected_column_data = None

    for column in row.index:
        if column == selected_column:
            selected_column_data = row[column]
        else:
            metadata_dict[column] = row[column]
    
    # print(selected_column_data)
    try:
        docs.extend(text_splitter.create_documents([selected_column_data],[metadata_dict]))
    except:
        print("This document did not get uploaded")
        print(selected_column_data)
        
        
    # break

This document did not get uploaded
nan
This document did not get uploaded
nan


In [27]:
docs[0].metadata

{'_id': '6436912a6f411a001ddba130',
 'source_id': 'LBA',
 'source_name': 'Reuters News',
 'title': 'Middle East Crude-Oman, Dubai rise on tighter supply outlook; Kuwait sets May OSP',
 'publishedAt_str': '2023-04-12T10:05:26.000Z'}

In [33]:
docs[0]

Document(page_content='SINGAPORE, April 12 (Reuters) - Middle East crude\nbenchmarks Oman and Dubai advanced for a third consecutive\nsession as the market expects tightening crude supply from the\nOPEC+ group from May when they start the additional output\nreduction.\n    Analysts from Energy Aspects expected some Chinese and South\nKorean refiners have nominated 450,000 barrels-per-day (bpd)\nless of crude oil supply from Saudi Aramco in May than their\ncontractual volume.\n    Aramco may also choose to lower oil throughput at its\ndomestic refineries to limit the amounts by which they must cut\ncrude exports, the analysts said.\n    Saudi Arabia plans to cut an extra of 500,000 bpd of crude\nproduction from May to the rest of the year. The top oil\nexporter was reported to supply full nominated volume to\nclients.', metadata={'_id': '6436912a6f411a001ddba130', 'source_id': 'LBA', 'source_name': 'Reuters News', 'title': 'Middle East Crude-Oman, Dubai rise on tighter supply outlook; K

In [23]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv("OPENAI_API_KEY"),
                model_name=os.getenv("EMBEDDINGS_MODEL")
                # model_name="text-embedding-ada-002",
                
            )
coll_n="Fianancial_News"
emb_fn=openai_ef

In [24]:
try:
    collection = client.get_collection(
                name=coll_n,
                embedding_function=emb_fn)
except ValueError:
    coll_l = [i.name for i in client.list_collections()]
    message = "Oops, it seems the collection doesn't exist. Please choose a collection from the list below or create a new one."
    # print(f"{message}\n\n{coll_l}")
    # return f"{message}\n\n{coll_l}"
    # return message, coll_l

In [30]:
from tqdm import tqdm
import uuid

In [31]:
for i in tqdm(docs, desc="Uploading and Indexing the Document",ncols=80, ascii=True):
# print("not found")
    collection.add(
        documents=[i.page_content],
        metadatas=[i.metadata],
        ids=[str(uuid.uuid4().int)]
        )

message = f"The document has been successfully uploaded to the database!"
# print(f"\n{message}")
# return f"\n{message}"

Uploading and Indexing the Document: 100%|##| 4288/4288 [29:04<00:00,  2.46it/s]


In [ ]:
docs=[]
for i in docs_1:
    docs.extend(text_splitter.create_documents([i.page_content],[i.metadata]))

In [21]:
from langchain.document_loaders import UnstructuredFileLoader
from unstructured.cleaners.core import clean_extra_whitespace

In [22]:
loader = UnstructuredFileLoader(file_path="raw_data/test_4.pdf", mode="paged",post_processors=[clean_extra_whitespace],strategy="auto")
docs_1 = loader.load()



/Users/y.abdalla/Desktop/Yousif/app_dev_server/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
for i in docs_1:
    print(i.metadata)
    break

{'source': 'raw_data/test_4.pdf', 'coordinates': {'points': ((72.0, 604.7760000000001), (72.0, 719.59872), (541.97964, 719.59872), (541.97964, 604.7760000000001)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'filename': 'test_4.pdf', 'file_directory': 'raw_data', 'last_modified': '2023-10-31T15:33:53', 'filetype': 'application/pdf', 'page_number': 1, 'links': [{'text': 'Click here to see a non - exhaustive list of Amazon trademarks .', 'url': 'https://www.amazon.com/gp/help/customer/display.html/?nodeId=200738910', 'start_index': 11}], 'parent_id': '53cf26fe2dc7b394fce25f79e2973c46'}


In [25]:
# # text splitter section
docs=[]
for i in docs_1:
    docs.extend(text_splitter.create_documents([i.page_content],[i.metadata]))

## splitiing the data into files 

In [5]:
data.head()

,_id,source_id,source_name,title,content,publishedAt_str
0,6436912a6f411a001ddba130,LBA,Reuters News,"Middle East Crude-Oman, Dubai rise on tighter ...","SINGAPORE, April 12 (Reuters) - Middle...",2023-04-12T10:05:26.000Z
1,64a687aec9a6f3002ebc7e72,LBA,Reuters News,Yellen to press for additional reforms at Worl...,In excerpts of her remarks obtained by Reuters...,2023-04-12T10:00:00.000Z
2,64a687a8c9a6f3002ebc7e14,MISTNW,Mist News,Masdar expands presence in US with completion ...,"The Big Beau project, located in California, c...",2023-04-12T09:25:20.000Z
3,64958417c9a6f3002e054a86,MISTNW,Mist News,DEWA CEO receives CEO of COP28,The two sides discussed ways to enhance cooper...,2023-04-12T09:09:18.000Z
4,64a6894dc9a6f3002ebc8dd6,EMNEAG,Emirates News Agency (WAM),COP28 President-Designate calls for ‘Fundament...,During a series of engagements with ministers ...,2023-04-12T00:00:00.000Z


In [13]:
import os
def save_content_as_text_files(data_frame, output_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Iterate through each row in the DataFrame
    for index, row in data_frame.iterrows():
        # Extract the content from the 'content' column
        content = row['content']

        # Define the file path based on the row number
        file_path = os.path.join(output_directory, f'{index + 1}.txt')
        try:
        # Write the content to the text file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(content)
        except:
            print(f"issue detected {content} file number {index}")


In [14]:
save_content_as_text_files(data_frame=data, output_directory="text_collection")

issue detected nan file number 382
issue detected nan file number 926


## Creating collection

In [18]:
import sys
sys.path.append("./scripts")
from doc_loader_chromadb import create_collection

create_collection(client=chroma_client,name="Fianancial_News",emb_fn=openai_ef,News_source="Reuters_News",News_category="Gulf")

'The Fianancial_News collection has been successfully created!'

In [5]:
for i in chroma_client.list_collections():
    print(i.name,i.metadata)

## Uploading to the collection

In [10]:
from doc_loader_chromadb import upload_to_collection_chromadb

upload_to_collection_chromadb(client=chroma_client,coll_n="HR_Department",doc_lo="./raw_data/test_2.txt",strategy="auto")

/Users/y.abdalla/Desktop/Yousif/app_dev/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading and Indexing the Document: 100%|########| 6/6 [00:03<00:00,  1.87it/s]


The file test_2 has been successfully uploaded to the database!


In [11]:
upload_to_collection_chromadb(client=chroma_client,coll_n="HR_Department",doc_lo="./raw_data/test_5.docx",strategy="auto")

Uploading and Indexing the Document: 100%|########| 1/1 [00:00<00:00,  1.52it/s]


The file test_5 has been successfully uploaded to the database!


## Deleting collection

In [14]:
collection = chroma_client.get_collection(name="HR_Department")
print("**Checking first Document**\n")
print(collection.peek()["documents"][0])
print("\n**Checking the last Document**\n")
print(collection.peek()["documents"][-1])
print("\nNumber of document current in the db")
print(collection.count())

**Checking first Document**

This July 4, 2023 was hot. Earth’s hottest day on record, in fact.

By Leo Sands Updated July 5, 2023 at 9:53 a.m. EDT|Published July 5, 2023 at 6:30 a.m. EDT

A man walks in front of an installation featuring a burning Earth at a park in Daegu, South Korea, on Wednesday. (Yonhap/EPA-EFE/Shutterstock)

Share

Comment 1002 Tuesday was the hottest day on Earth since at least 1979, with the global average temperature reaching 62.92 degrees Fahrenheit (17.18 degrees Celsius), according to data from the U.S. National Centers for Environmental Prediction.

Want to know how your actions can help make a difference for our planet? Sign up for the Climate Coach newsletter, in your inbox every Tuesday and Thursday. As a result, some scientists believe July 4 may have been one of the hottest days on Earth in about 125,000 years, due to a dangerous combination of climate change causing global temperatures to soar, the return of the El Niño pattern and the start of summe

In [2]:
for i in chroma_client.list_collections():
    print(i)

name='HR_department' id=UUID('ca82f02b-311a-4734-87b7-51654e612457') metadata={'Yousif': 'profile'}


In [9]:
from doc_loader_chromadb import delete_collection

delete_collection(client=chroma_client,coll_na="HR_department")

'The collection has been successfully deleted.'

In [10]:
for i in chroma_client.list_collections():
    print(i)

## Testing the chat funtion

In [3]:
for i in chroma_client.list_collections():
    print(i)

name='HR_department' id=UUID('ca82f02b-311a-4734-87b7-51654e612457') metadata={'Yousif': 'profile'}


In [4]:
import sys
sys.path.append("./scripts")
from Chat_openai_yousif import chat_f

re,doc = chat_f(chroma_client,collection="HR_department",question ="what is the information required when I have a copyright concern?")
# re,doc= chat_f(chroma_client,collection="HR_department",question ="do you remember anything we discuss before ?")

In [6]:
print(re)

When you have a copyright concern, the following information is typically required:

1. A description of the copyrighted work that has been infringed upon.
2. The location on the site where the infringing material is located.
3. Your address, telephone number, and email address.
4. A statement by you that you have a good-faith belief that the disputed use is not authorized by the copyright owner, its agent, or the law.
5. A statement by you, made under penalty of perjury, that the information provided in your notice is accurate and that you are the copyright owner or authorized to act on the copyright owner's behalf.

You can submit your complaint using the online form provided by the platform. Alternatively, for copyright complaints only, you may submit written claims of copyright infringement to the Copyright Agent at Amazon.com's Legal Department. The contact details for the Copyright Agent are as follows:

Copyright Agent
Amazon.com Legal Department
P.O. Box 81226
Seattle, WA 98108

In [11]:
{"response":re,"retrieved_doc":doc}

{'response': "When you have a copyright concern, the following information is typically required:\n\n1. A description of the copyrighted work that has been infringed upon.\n2. The location on the site where the infringing material is located.\n3. Your address, telephone number, and email address.\n4. A statement by you that you have a good-faith belief that the disputed use is not authorized by the copyright owner, its agent, or the law.\n5. A statement by you, made under penalty of perjury, that the information provided in your notice is accurate and that you are the copyright owner or authorized to act on the copyright owner's behalf.\n\nYou can submit your complaint using the online form provided by the platform. Alternatively, for copyright complaints only, you may submit written claims of copyright infringement to the Copyright Agent at Amazon.com's Legal Department. The contact details for the Copyright Agent are as follows:\n\nCopyright Agent\nAmazon.com Legal Department\nP.O. B

In [12]:
doc

[Document(page_content='A description of the copyrighted work that you claim has been infringed upon; • A description of where the material that you claim is infringing is located on the site; • Your address, telephone number, and e-mail address; • A statement by you that you have a good-faith belief that the disputed use is not authorized by the copyright owner, its agent, or the law; and', metadata={'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 6}),
 Document(page_content="A statement by you, made under penalty of perjury, that the above information in your notice is accurate and that you are the copyright owner or authorized to act on the copyright owner's behalf.", metadata={'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 7}),
 Document(page_content='NOTICE AND PROCEDURE FOR MAKING CLAIMS OF INTELLECTUAL PROPERTY INFRINGEMENT If you believe that your intellectual property rights have been infringed, please submit your complaint usin

In [14]:
doc_2 = [do.to_json() for do in doc]

In [21]:
for i in doc_2:
    print(i["kwargs"]["metadata"])

{'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 6}
{'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 7}
{'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 6}
{'filename': 'test_4.pdf', 'filetype': 'application/pdf', 'page_number': 3}
